# Video 2: Using a notebook client

#### Step 0: Installation and imports

In [ ]:
# Install necessary libraries
!pip install caterva2 blosc2 blosc2_grok matplotlib

In [1]:
# Imports
import time

import blosc2
import matplotlib.pyplot as plt
import numpy as np

import caterva2 as cat2

#### Step 1: Setup client and get root

In [2]:
client = cat2.Client("https://cat2.cloud/demo", ("user@example.com", "foobar11"))
client.get_roots()

{'@public': {'name': '@public', 'http': '', 'subscribed': True},
 '@personal': {'name': '@personal', 'http': '', 'subscribed': True},
 '@shared': {'name': '@shared', 'http': '', 'subscribed': True}}

In [3]:
myroot = client.get("@public")
print(myroot)
myroot.file_list

@public


['examples/README.md',
 'examples/Wutujing-River.jpg',
 'examples/cat2cloud-brochure.pdf',
 'examples/dir1/ds-2d.b2nd',
 'examples/dir1/ds-3d.b2nd',
 'examples/dir2/ds-4d.b2nd',
 'examples/ds-1d-b.b2nd',
 'examples/ds-1d-fields.b2nd',
 'examples/ds-1d.b2nd',
 'examples/ds-2d-fields.b2nd',
 'examples/ds-hello.b2frame',
 'examples/ds-sc-attr.b2nd',
 'examples/gaia-ly.b2nd',
 'examples/lazyarray-large.png',
 'examples/lung-jpeg2000_10x.b2nd',
 'examples/numbers_color.b2nd',
 'examples/numbers_gray.b2nd',
 'examples/sa-1M.b2nd',
 'examples/tomo-guess-test.b2nd']

#### Step 2: Access and visualise a file

In [ ]:
myfile = "lung-jpeg2000_10x.b2nd"
address = "examples/" + myfile
ds = myroot[address]
print(ds)
print(f"Dataset shape is {ds.shape} and dtype is {ds.dtype}")

In [ ]:
nbytes = client.get_info(myroot.name + "/" + address)["schunk"]["cparams"]["typesize"]

# Download and decompress slice of dataset
t1 = time.time()
temp = ds[5:9]  # download and decompress slice of dataset
t2 = time.time()
dltime1 = t2 - t1

# Download compressed slice of dataset
t1 = time.time()
temp = ds.slice(slice(5, 9))  # download compressed slice of dataset
t2 = time.time()
dltime2 = t2 - t1

numKB = np.prod(temp.shape) * nbytes / (2**10)
print(f"Download time with ds[5:9]: {round(dltime1, 2)} s -- dataset size {numKB} KB")
print(f"Download time with ds.slice(slice(5,9)): {round(dltime2, 2)} s  -- dataset size {numKB} KB")
plt.imshow(temp[0])
plt.gca().axis("off");

#### Step 3: Download and save uncompressed file locally

In [ ]:
client.download(myroot.name + "/" + address, myfile)
arr = blosc2.open(myfile)[:]
newarr = blosc2.asarray(arr, cparams={"clevel": 0})
mylocalfile = "uncompressed_" + myfile
blosc2.save(newarr, mylocalfile, cparams={"clevel": 0}, mode="w")

#### Step 4: Upload uncompressed file to server

In [5]:
mylocalfile = "C:\\Users\\lshaw\Downloads\\tomo_00068.h5"
uploadaddress = "examples/" + "bigfile"
client.upload(mylocalfile, myroot.name + "/" + uploadaddress)
myroot.file_list

RemoteProtocolError: Server disconnected without sending a response.

#### Step 5: Access and visualise uncompressed file

In [ ]:
ds = myroot[uploadaddress]
print(ds)
t1 = time.time()
temp = ds[5:9]
t2 = time.time()
totTime = t2 - t1
print(f"Download time: {round(totTime, 2)} s")
plt.imshow(temp[0])  # try 1 and 2
plt.gca().axis("off");

#### Step 6: Remove uncompressed file from server

In [ ]:
client.remove(myroot.name + "/" + uploadaddress)